In [1]:
import os,sys
from pathlib import Path
from typing import Any,List, Set, Dict, Tuple, Optional, Iterable, Mapping, Union, Callable, TypeVar, cast

import time
from datetime import datetime
from collections import OrderedDict
from functools import partial

# data or cache IO
import json
import joblib

# print, debug
from pprint import pprint
from IPython.display import display, HTML # for pretty-print pandas dataframe
from IPython.core.debugger import set_trace

sys.dont_write_bytecode = True



# numpy and friends
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image


# torch imports
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from  torch.linalg import norm as tnorm
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import datasets, transforms
import torchvision.utils as vutils

# Pytorch-lightning
import pytorch_lightning as pl
from pytorch_lightning.core  import LightningModule
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.tuner.tuning import Tuner
from pytorch_lightning.callbacks import Callback

# Select Visible GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
print("ID of assigned GPU: ", os.environ["CUDA_VISIBLE_DEVICES"])
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Reprlearn
from reprlearn.visualize.utils import get_fig, show_timg, show_timgs, show_npimgs, show_batch 
from reprlearn.utils.misc import info, now2str, get_first_img_info, count_imgs

# import artifact compute functions
from reprlearn.utils.fpts import estimate_projection, compute_artifacts, compute_artifact
from reprlearn.utils.fpts import estimate_projection_fp #, compute_artifacts, compute_artifact

# GLOBALS:
# === Path to experiment dataset (GM256)
SSD_HOME = Path(os.environ.get('myssd', '~'))
TMPDIR = Path(os.environ['TMPDIR'])
              
# -- Move data to cluster's compute node
# srcdir = SSD_HOME / 'Datasets/gm256_from_arya'
# dstdir = TMPDIR 
# !rsync -aznP {str(srcdir)} {str(dstdir)}

# -- Data root-dir in compute node
# DATA_ROOT = SSD_HOME / 'Datasets/gm256_from_arya'    # if rsync is not yet done
DATA_ROOT = TMPDIR / 'gm256_from_arya'                 # elif rsync is done
REAL_DATA_DIR = DATA_ROOT / 'real-celebahq256'

# === Verify
assert DATA_ROOT.exists()
# print('Subdirs: ')
# !ls {DATA_ROOT}

# print('\nNum. real datapts: ')
# !ls {REAL_DATA_DIR} | wc -l



ID of assigned GPU:  0


In [12]:
# concat df's
# train 

df_dir = Path('./Outputs/dfs_with_proj_rgb_compiled_20230428/')

df_train_fps = []
for fp in df_dir.iterdir():
    if fp.is_file() and fp.name.startswith('df-train-'):
        df_train_fps.append(fp)
        
print(len(df_train_fps))
df_train_fps = sorted(df_train_fps)
[print(fp.name) for fp in df_train_fps];



19
df-train-0-1297-with-proj-rgb-gm256-onek-from-15-a-artRGB-trainset-20230427-123205.csv
df-train-1000-1599-with-proj-rgb-gm256-onek-20230427-150811.csv
df-train-10000-10499-with-proj-rgb-gm256-onek-20230427-144225.csv
df-train-10500-10549-with-proj-rgb-gm256-onek-20230428-041323.csv
df-train-11000-11899-with-proj-rgb-gm256-onek-20230428-092220.csv
df-train-12000-12499-with-proj-rgb-gm256-onek-20230427-145937.csv
df-train-12500-12699-with-proj-rgb-gm256-onek-20230428-063306.csv
df-train-13000-13499-with-proj-rgb-gm256-onek-20230428-063844.csv
df-train-2000-2099-with-proj-rgb-gm256-onek-20230428-010752.csv
df-train-3000-3499-with-proj-rgb-gm256-onek-20230427-135603.csv
df-train-3500-3549-with-proj-rgb-gm256-onek-20230428-040816.csv
df-train-4000-4999-with-proj-rgb-gm256-onek-20230427-025352.csv
df-train-5000-5499-with-proj-rgb-gm256-onek-20230427-141205.csv
df-train-5500-5549-with-proj-rgb-gm256-onek-20230428-040935.csv
df-train-6000-6499-with-proj-rgb-gm256-onek-20230427-142139.csv
df